<a href="https://colab.research.google.com/github/BaekJeongsuk/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_4%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 오픈소스 기반 데이터 분석 4강 - 데이터 수집


## 4-1 CSV 파일 읽기

In [ ]:
import pandas as pd

## data.csv 파일 읽기
df = pd.read_csv('data.csv', encoding = 'utf-8', sep=',', header = 0, index_col=None, skiprows=None, nrows=None)
print(df)

           날짜    체중  골격근량  체지방량
0  2025.02.06  64.7  30.0  11.1
1  2025.02.04  64.0  29.3  11.6


## 4-2 JSON 파일 읽기



In [ ]:
import json
import pandas as pd

## data.json 파일 출력
with open('data.json', mode='r', encoding='utf-8') as f:
    data = json.load(f)
print(data)

## data.json 파일 DataFrame 읽기
df = pd.read_json('data.json', orient='records', encoding='utf-8', lines=False)
print(df)

{'매출데이터': [{'월': '2025-01', '매출액': 1000000, '비용': 700000, '이익': 300000}, {'월': '2025-02', '매출액': 1200000, '비용': 800000, '이익': 400000}, {'월': '2025-03', '매출액': 1500000, '비용': 900000, '이익': 600000}]}
                                               매출데이터
0  {'월': '2025-01', '매출액': 1000000, '비용': 700000,...
1  {'월': '2025-02', '매출액': 1200000, '비용': 800000,...
2  {'월': '2025-03', '매출액': 1500000, '비용': 900000,...


## 4-3 텍스트 파일 읽기 및 데이터 추출

In [ ]:
import re  # 정규 표현식(Regular Expression)

## 파일(callcenter20250301.log) 오픈 및 읽기
with open('callcenter20250301.log', 'r', encoding='utf-8') as f:
    content = f.read()
## 주민등록번호 패턴 생성
pattern = re.compile(r'(\d{6})-(\d{7})')

## 주민등록번호 마스킹
masked_content = pattern.sub(r'\1-*******', content)

## 마스킹된 파일(callcenter20250301_masked.log) 오픈 및 쓰기
with open('callcenter20250301_masked.log', mode='w') as f:
    f.write(masked_content)

print("주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.")

주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.


## 4-4 Open-Meteo의 무료 날씨 API를 통한 특정 지역 온도 조회

In [ ]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m"
params = {
    "latitude": "35.80603",
    "longitude": "128.53391",
    "current": "temperature_2m"
}

try:
    ## URL 및 파라미터 전송
    response = requests.get(url, params=params)
    response.raise_for_status()

    ## JSON 데이터 읽기
    data = response.json()

    print("API 응답:", data)
    print(f"대구시 달서구의 현재 온도는 : {data['current']['temperature_2m']}{data['current_units']['temperature_2m']} 입니다.")
    ## print("대구시 달서구의 현재 온도는 : {0}{1} 입니다.".format(data['current']['temperature_2m'], data['current_units']['temperature_2m']))
except requests.exceptions.RequestException as e:
    print(f"API 호출 실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

API 응답: {'latitude': 35.8, 'longitude': 128.5625, 'generationtime_ms': 0.0362396240234375, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 59.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-04T17:45', 'interval': 900, 'temperature_2m': 18.5}}
대구시 달서구의 현재 온도는 : 18.5°C 입니다.


## 4-5 Selenium과 lxml을 이용한 웹 스크래핑

In [ ]:
!curl -o google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb -y
!pip install selenium webdriver_manager

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0   269M      0 --:--:-- --:--:-- --:--:--  269M
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.9 MB/131 MB of archives.
After this operation, 448 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libvulkan1 amd64 1.3.204.1-2 [128 kB]
Get:2 /content/google-chrome-stable_current_amd64.deb google-chrome-stable amd64 141.0.7390.54-1 [121 MB]
Get:3 http://ar

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from lxml import html
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')               # 브라우저 창 없이 실행
chrome_options.add_argument('--no-sandbox')             # 보안모드 비활성화 (Colab 필수)
chrome_options.add_argument('--disable-dev-shm-usage')  # 메모리 부족 방지 (Colab 필수)
chrome_options.add_argument('--window-size=1920x1080')  # 창 크기 설정(가상)
chrome_options.add_argument('--disable-gpu')            # GPU 가속 비활성화 (일부 환경 안정성)
chrome_options.binary_location = "/usr/bin/google-chrome-stable"  # Colab용 크롬 경로 지정

## 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)
## 사이트 접속
url = 'http://professor.knou.ac.kr/jaehwachung/index.do'
driver.get(url)
## 사이트 접속 대기
time.sleep(2)
## 페이지 제목 출력
page_source = driver.page_source
tree = html.fromstring(page_source)

title_text = tree.xpath('//title/text()')
print(title_text)

## 드라이버 종료
driver.quit()

['\n\t\tAlert \n\t\t\n\t']



# 실습 시나리오

## 공공데이터 포털 가입 및 데이터 신청

- [https://www.data.go.kr](https://www.data.go.kr)
- 한국환경공단 에어코리아 대기오염정보 데이터 신청

In [ ]:
import requests

## 데이터 수집 url 및 api key 설정
url = 'http://openapi.seoul.go.kr:8088/72557745436261653436656d5a764d/json/TbeEnergyFueltotalamt/1/5'
api_key = '72557745436261653436656d5a764d'

params = {
    'serviceKey': api_key,
    'returnType': 'json',
    'numOfRows': '100',
    'pageNo': '1',
    'sidoName': '서울',
    'ver': '1.0'
}

## 데이터 수집
response = requests.get(url, params=params)

if response.status_code == 200:
    print("api 호출 성공")
    print(response.json())
else:
    print(f"API 호출 실패: {response.status_code}")

## 호출 성공/실패 출력
## print(response.json())


api 호출 성공
{'RESULT': {'CODE': 'ERROR-500', 'MESSAGE': '서버 오류입니다.\n지속적으로 발생시 열린 데이터 광장으로 문의(Q&A) 바랍니다.'}}


In [ ]:
import requests

url = "http://openapi.seoul.go.kr:8088/72557745436261653436656d5a764d/json/energyUseDataSummaryInfo/1/5/2015/01"
api_key = '72557745436261653436656d5a764d'

params = {
    'serviceKey': api_key,
    'returnType': 'json',
    'numOfRows': '100',
    'pageNo': '1',
    'sidoName': '서울',
    'ver': '1.0'
}

response = requests.get(url, params=params)

if response.status_code == 200:
    print("api 호출 성공")
    print(response.json())
else:
    print(f"API 호출 실패: {response.status_code}")


api 호출 성공
{'energyUseDataSummaryInfo': {'list_total_count': 7, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'CNT': '767791', 'EUS': '193784708', 'EUS1': '194781915', 'EUS2': '204969429', 'ECO2_1': '-6090964', 'ECO2_2': '-2582568.736', 'GUS': '59133720', 'GUS1': '57163993', 'GUS2': '68297619', 'GCO2_1': '-3597086', 'GCO2_2': '-8057472.64', 'WUS': '12819757.886', 'WUS1': '12723680.426', 'WUS2': '12899476.73', 'WCO2_1': '8179.308', 'WCO2_2': '2715.530256', 'HUS': '22740838.937', 'HUS1': '23400055.303', 'HUS2': '27090493.875', 'HCO2_1': '-2504435.652', 'HCO2_2': '-33660084.213069', 'REG_DATE': '2015-06-04 17:03:55.0'}, {'YEAR': '2015', 'MON': '01', 'MM_TYPE': '학교', 'CNT': '1382', 'EUS': '134955565', 'EUS1': '128707423', 'EUS2': '145561511', 'ECO2_1': '-2178902', 'ECO2_2': '-923854.448', 'GUS': '9107197', 'GUS1': '8410968', 'GUS2': '10745416', 'GCO2_1': '-470995', 'GCO2_2': '-1055028.8', 'WUS': '2075819.2', 'WUS1': '2097433.

In [ ]:
import requests

api_key = '72557745436261653436656d5a764d'

params = {
    'serviceKey': api_key,
    'returnType': 'json',
    'numOfRows': '100',
    ## 'pageNo': '1',
    'sidoName': '서울',
    'ver': '1.0'
}

for YEAR in range(2015, 2025):
    for MON in range(1, 13):
        url = f"http://openapi.seoul.go.kr:8088/72557745436261653436656d5a764d/json/energyUseDataSummaryInfo/1/5/{YEAR}/{MON}"
        response = requests.get(url, params=params)

if response.status_code == 200:
    print(f"{YEAR}-{MON} api 호출 성공")
    print(response.json())
else:
    print(f"{YEAR}-{MON} API 호출 실패: {response.status_code}")



2024-12 api 호출 성공
{'energyUseDataSummaryInfo': {'list_total_count': 7, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2024', 'MON': '12', 'MM_TYPE': '공동주택관리소', 'CNT': '3716', 'EUS': '548480228', 'EUS1': '557799290', 'EUS2': '559693904', 'ECO2_1': '-10266369', 'ECO2_2': '-10266369', 'GUS': '8221238', 'GUS1': '9759209', 'GUS2': '9093999', 'GCO2_1': '-1114991.5', 'GCO2_2': '-2229983', 'WUS': '20262147.7', 'WUS1': '20228097.5', 'WUS2': '20488646.8', 'WCO2_1': '-96224.45', 'WCO2_2': '-288673.35', 'HUS': '0', 'HUS1': '0', 'HUS2': '0', 'HCO2_1': '0', 'HCO2_2': '0', 'REG_DATE': '2025-03-01 04:02:55.0'}, {'YEAR': '2024', 'MON': '12', 'MM_TYPE': '기업', 'CNT': '19451', 'EUS': '633735769', 'EUS1': '641412053', 'EUS2': '669494932', 'ECO2_1': '-21717723.5', 'ECO2_2': '-21717723.5', 'GUS': '30096486', 'GUS1': '45742965', 'GUS2': '34384677', 'GCO2_1': '-9960022.5', 'GCO2_2': '-19920045', 'WUS': '4448777.4', 'WUS1': '4406465.4', 'WUS2': '4396396.4', 'WCO2_1': '47346.5', 'WCO2

In [ ]:
response.json()

{'energyUseDataSummaryInfo': {'list_total_count': 7,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'YEAR': '2024',
    'MON': '12',
    'MM_TYPE': '공동주택관리소',
    'CNT': '3716',
    'EUS': '548480228',
    'EUS1': '557799290',
    'EUS2': '559693904',
    'ECO2_1': '-10266369',
    'ECO2_2': '-10266369',
    'GUS': '8221238',
    'GUS1': '9759209',
    'GUS2': '9093999',
    'GCO2_1': '-1114991.5',
    'GCO2_2': '-2229983',
    'WUS': '20262147.7',
    'WUS1': '20228097.5',
    'WUS2': '20488646.8',
    'WCO2_1': '-96224.45',
    'WCO2_2': '-288673.35',
    'HUS': '0',
    'HUS1': '0',
    'HUS2': '0',
    'HCO2_1': '0',
    'HCO2_2': '0',
    'REG_DATE': '2025-03-01 04:02:55.0'},
   {'YEAR': '2024',
    'MON': '12',
    'MM_TYPE': '기업',
    'CNT': '19451',
    'EUS': '633735769',
    'EUS1': '641412053',
    'EUS2': '669494932',
    'ECO2_1': '-21717723.5',
    'ECO2_2': '-21717723.5',
    'GUS': '30096486',
    'GUS1': '45742965',
    'GUS2': '34384677',
    'G

In [ ]:

import pandas as pd

## requests로부터 response, body, item 항목 읽기
data = response.json()['energyUseDataSummaryInfo']['row']
## print(type(data))
## print(len(data))
## print(data[0])

df = pd.DataFrame(data)
df

,YEAR,MON,MM_TYPE,CNT,EUS,EUS1,EUS2,ECO2_1,ECO2_2,GUS,...,WUS1,WUS2,WCO2_1,WCO2_2,HUS,HUS1,HUS2,HCO2_1,HCO2_2,REG_DATE
0,2024,12,공동주택관리소,3716,548480228,557799290,559693904,-10266369,-10266369,8221238,...,20228097.5,20488646.8,-96224.45,-288673.35,0,0,0,0,0,2025-03-01 04:02:55.0
1,2024,12,기업,19451,633735769,641412053,669494932,-21717723.5,-21717723.5,30096486,...,4406465.4,4396396.4,47346.5,142039.5,0,0,0,0,0,2025-03-01 04:02:55.0
2,2024,12,개인,1250620,134828126,138280636,138879530,-3751957,-3504709,42103424,...,7440605,7625718.9,-206308.35,-573725.128,1513199.277,1598857.184,1781397.386,-176928.008,-376996.699240516,2025-03-01 04:02:55.0
3,2024,12,공공기관,2459,152992881,161100765,162060905,-8587954,-8587954,3814815,...,854671.2,833317.4,52847.7,158543.1,0,0,0,0,0,2025-03-01 04:02:55.0
4,2024,12,학교,6742,164092999,170867152,169660376,-6170678,-6172770.32,4362563,...,1898794,1939698.4,-60405.2,-181607.796,1062.62,902.03,1220.91,1.15,3.3315,2025-03-01 04:02:55.0
